<a href="https://colab.research.google.com/github/SarahLares/Coursera_Capstone/blob/master/Clustering_Neighborhoods_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Segmenting and Clustering Neighborhoods in Toronto


##Part 1: Preprocessing of Toronto Data

### Import Libraries

In [280]:
import numpy as np 
import pandas as pd
from geopy.geocoders import Nominatim #
import folium # map rendering library
from pandas import json_normalize # tranform JSON file into a pandas dataframe
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

### Importing the Data as DataFrame

In [248]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(url)
df_toronto = pd.read_json(df[0].to_json(orient='records'))

### Delete the intances whit **Borough = Not assigned**

In [249]:
df_toronto = df_toronto.loc[df_toronto['Borough']!='Not assigned'].reset_index(drop=True)

In [250]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


We can see that the data have grouped the Neighbourhood whit the same Postal Code

In [251]:
df_toronto.loc[df_toronto['Neighbourhood']=='Not assigned']

,Postal Code,Borough,Neighbourhood


The data don't have **Neighbourhood = Not assigned**

### Shape of the data

In [252]:
df_toronto.shape

(103, 3)

## Part 2: Get Latitude and Longitude for Neighbourhood

In [253]:
url_geo = 'http://cocl.us/Geospatial_data'
df_geo = pd.read_csv(url_geo)

In [254]:
df_toronto = df_toronto.merge(df_geo, left_on='Postal Code', right_on='Postal Code')

In [255]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [256]:
df_toronto['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

## Part 3: Clustering Segmentation of Toronto  Neighbourhoods

In [257]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronti are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronti are 43.6534817, -79.3839347.


### Map of Toronto and their Borough

In [258]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### We will work with only boroughs that contain the word Toronto

In [259]:
toronto_data = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(39, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [260]:
# create map of Manhattan using latitude and longitude values
map_toronto_2 = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'],toronto_data['Longitude'],toronto_data['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_2)  
    
map_toronto_2

###Define Foursquare Credentials and Version

In [261]:
CLIENT_ID = '41KH5UCUZIRFBZDRECEWCZIVIKUQZIH5ABT2TJ0LXVE4QLMZ'
CLIENT_SECRET = 'OLQMMY3ROCH41CJFYDCV4DKZCBRS01SCRV5XGNZCU1F4HWIA' 

VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 41KH5UCUZIRFBZDRECEWCZIVIKUQZIH5ABT2TJ0LXVE4QLMZ
CLIENT_SECRET:OLQMMY3ROCH41CJFYDCV4DKZCBRS01SCRV5XGNZCU1F4HWIA


We will work with the Top venues in each Neighbourhood (for postal code) within 500 meters radius

In [262]:
LIMIT = 20
radius = 500 

###Analyze Each Neighborhood

In [263]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([( name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [264]:
# type your answer here

Toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )




Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [265]:
Toronto_venues['Neighborhood'].unique().shape

(39,)

In [266]:
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

toronto_onehot.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,...,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [267]:
toronto_onehot['Neighborhood'].unique().shape

(39,)

In [268]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,...,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.050000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.050000,0.000000,0.000000,0.05,0.05,0.00,0.00,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,0.10,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.05,0.000000,0.00,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.050000,0.000000,0.050000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.100000,0.000000,0.00,0.00,0.050000,0.00,0.000000,0.150000,0.00,0.000000,0.00,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.050000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.05,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.050000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.055556,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.055556,0.00,0.00,0.055556,0.00,0.055556,0.000000,0.00,0.000000,0.00,...,0.00,0.00,0.000000,0.00,0.055556,0.000000,0.055556,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.055556,0.00,0.00,0.00,0.055556,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.058824,0.058824,0.058824,0.117647,0.176471,0.058824,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.058824,0.00,0.00,0.00,0.00,0.00,0.058824,0.000000,0.058824,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,...,0.00,0.00,0.000000,0.00,0.000000,0.058824,0.000000,0.00,0.000000,0.000000,0.058824,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.05,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,...,0.05,0.00,0.000000,0.05,0.000000,0.000000,0.000000,0.00,0.000000,0.050000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.050000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.050000,0.00,0.00,0.00,0

In [269]:
toronto_onehot.describe()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,...,Poke Place,Portuguese Restaurant,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
count,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,...,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000,647.000000
mean,0.001546,0.001546,0.001546,0.003091,0.004637,0.001546,0.010819,0.001546,0.012365,0.001546,0.004637,0.004637,0.001546,0.001546,0.003091,0.001546,0.001546,0.023184,0.004637,0.017002,0.001546,0.012365,0.001546,0.001546,0.001546,0.001546,0.012365,0.001546,0.012365,0.009274,0.004637,0.004637,0.007728,0.001546,0.003091,0.068006,0.001546,0.001546,0.003091,0.003091,...,0.001546,0.001546,0.017002,0.004637,0.001546,0.001546,0.040185,0.001546,0.003091,0.020093,0.001546,0.010819,0.001546,0.001546,0.001546,0.001546,0.001546,0.006182,0.003091,0.004637,0.001546,0.001546,0.001546,0.007728,0.001546,0.004637,0.017002,0.001546,0.001546,0.001546,0.010819,0.007728,0.001546,0.001546,0.006182,0.001546,0.007728,0.004637,0.003091,0.009274
std,0.039314,0.039314,0.039314,0.055555,0.067988,0.039314,0.103531,0.039314,0.110593,0.039314,0.067988,0.067988,0.039314,0.039314,0.055555,0.039314,0.039314,0.150604,0.067988,0.129377,0.039314,0.110593,0.039314,0.039314,0.039314,0.039314,0.110593,0.039314,0.110593,0.095926,0.067988,0.067988,0.087636,0.039314,0.055555,0.251951,0.039314,0.039314,0.055555,0.055555,...,0.039314,0.039314,0.129377,0.067988,0.039314,0.039314,0.196546,0.039314,0.055555,0.140426,0.039314,0.103531,0.039314,0.039314,0.039314,0.039314,0.039314,0.078445,0.055555,0.067988,0.039314,0.039314,0.039314,0.087636,0.039314,0.067988,0.129377,0.039314,0.039314,0.039314,0.103531,0.087636,0.039314,0.039314,0.078445,0.039314,0.087636,0.067988,0.055555,0.095926
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

###Let's print each neighborhood along with the top 5 most common venues

In [270]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0      Farmers Market  0.10
1  Seafood Restaurant  0.10
2          Restaurant  0.05
3        Liquor Store  0.05
4        Concert Hall  0.05


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.15
1     Coffee Shop  0.10
2  Breakfast Spot  0.10
3          Bakery  0.05
4   Grocery Store  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.11
1  Fast Food Restaurant  0.06
2           Pizza Place  0.06
3  Gym / Fitness Center  0.06
4                  Park  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0     Airport Service  0.18
1      Airport Lounge  0.12
2             Airport  0.06
3                 Bar  0.06
4  Airport Food Court  0.06


----Central Bay Street----
                venue  f

###Let's put that into a pandas dataframe

In [271]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [272]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Farmers Market,Seafood Restaurant,Beer Bar,Cocktail Bar,Coffee Shop,Museum,Concert Hall,Liquor Store,Restaurant,Bistro
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Yoga Studio,Stadium,Grocery Store,Gym,Italian Restaurant,Performing Arts Venue,Climbing Gym
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Garden,Butcher,Skate Park,Brewery,Restaurant,Spa,Recording Studio,Burrito Place,Farmers Market
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport,Plane,Boutique,Boat or Ferry,Coffee Shop,Rental Car Location,Bar,Harbor / Marina
4,Central Bay Street,Coffee Shop,Italian Restaurant,Art Museum,Sandwich Place,Spa,Bubble Tea Shop,Ramen Restaurant,Comic Shop,Japanese Restaurant,Poke Place


##Cluster Neighborhoods

In [273]:
from sklearn.cluster import KMeans

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([1, 4, 1, 1, 4, 4, 1, 4, 1, 1], dtype=int32)

add clustering labels to the dataframe

In [274]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Farmers Market,Seafood Restaurant,Beer Bar,Cocktail Bar,Coffee Shop,Museum,Concert Hall,Liquor Store,Restaurant,Bistro
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Yoga Studio,Stadium,Grocery Store,Gym,Italian Restaurant,Performing Arts Venue,Climbing Gym
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Garden,Butcher,Skate Park,Brewery,Restaurant,Spa,Recording Studio,Burrito Place,Farmers Market
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport,Plane,Boutique,Boat or Ferry,Coffee Shop,Rental Car Location,Bar,Harbor / Marina
4,Central Bay Street,Coffee Shop,Italian Restaurant,Art Museum,Sandwich Place,Spa,Bubble Tea Shop,Ramen Restaurant,Comic Shop,Japanese Restaurant,Poke Place


In [275]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [276]:
toronto_data.shape

(39, 5)

In [277]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Berczy Park,Farmers Market,Seafood Restaurant,Beer Bar,Cocktail Bar,Coffee Shop,Museum,Concert Hall,Liquor Store,Restaurant,Bistro
1,4,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Yoga Studio,Stadium,Grocery Store,Gym,Italian Restaurant,Performing Arts Venue,Climbing Gym
2,1,"Business reply mail Processing Centre, South C...",Light Rail Station,Garden,Butcher,Skate Park,Brewery,Restaurant,Spa,Recording Studio,Burrito Place,Farmers Market
3,1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport,Plane,Boutique,Boat or Ferry,Coffee Shop,Rental Car Location,Bar,Harbor / Marina
4,4,Central Bay Street,Coffee Shop,Italian Restaurant,Art Museum,Sandwich Place,Spa,Bubble Tea Shop,Ramen Restaurant,Comic Shop,Japanese Restaurant,Poke Place


In [278]:
toronto_merged = toronto_data
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted, lsuffix='Neighborhood', rsuffix='Neighborhood')

toronto_merged.head(40) 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Berczy Park,Farmers Market,Seafood Restaurant,Beer Bar,Cocktail Bar,Coffee Shop,Museum,Concert Hall,Liquor Store,Restaurant,Bistro
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Yoga Studio,Stadium,Grocery Store,Gym,Italian Restaurant,Performing Arts Venue,Climbing Gym
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,"Business reply mail Processing Centre, South C...",Light Rail Station,Garden,Butcher,Skate Park,Brewery,Restaurant,Spa,Recording Studio,Burrito Place,Farmers Market
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport,Plane,Boutique,Boat or Ferry,Coffee Shop,Rental Car Location,Bar,Harbor / Marina
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Central Bay Street,Coffee Shop,Italian Restaurant,Art Museum,Sandwich Place,Spa,Bubble Tea Shop,Ramen Restaurant,Comic Shop,Japanese Restaurant,Poke Place
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,Christie,Grocery Store,Café,Park,Coffee Shop,Italian Restaurant,Baby Store,Restaurant,Athletics & Sports,Diner,Candy Store
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Church and Wellesley,Creperie,Bookstore,Japanese Restaurant,Salon / Barbershop,Beer Bar,Breakfast Spot,Diner,Restaurant,Dance Studio,Bubble Tea Shop
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4,"Commerce Court, Victoria Hotel",Café,Gastropub,Japanese Restaurant,Restaurant,Coffee Shop,Pub,American Restaurant,Art Gallery,Gym,Gym / Fitness Center
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1,Davisville,Dessert Shop,Gym,Sushi Restaurant,Italian Restaurant,Toy / Game Store,Pizza Place,Thai Restaurant,Café,Sandwich Place,Park
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,Davisville North,Pizza Place,Park,Department Store,Breakfast Spot,Sandwich Place,Gym,Gym / Fitness Center,Food & Drink Shop,Dance Studio,Hotel


## Let's See The Clusters

In [284]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters